In [ ]:
import redivis

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
## basic functionality
import pandas as pd
import numpy as np
import re
import os
#import plotnine
#from plotnine import *
import geopandas as gpd
## repeated printouts
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import folium
from folium import Map
from folium.plugins import HeatMap


import datetime
import time


import census
import requests


In [ ]:
#Write in dependencies
'''
pip install census

OR

conda init
conda create -n env_dc
conda activate env_dc
conda install census

'''

In [ ]:
# Full documentation available at:
# https://apidocs.redivis.com/client-libraries/redivis-python

# load energy BENCHMARK data
table = redivis.table("table_1:6add")  # Reference any table in this project
dfb = table.to_dataframe()      # Load table as a dataframe

dfb.head()

# Load energy PERFORMANCE data

table = redivis.table("building_energy_performance:shjg")  # Reference any table in this project
dfp = table.to_dataframe()      # Load table as a dataframe

dfp.head()


In [ ]:
dfb.PID.nunique()

## Plot - geo - points (building energy/emissions)

First, subset to required year.

In [ ]:
from shapely.geometry import Point
from geopandas import GeoDataFrame

gdfb=dfb.copy()

geometry = [Point(xy) for xy in zip(gdfb['LONGITUDE'], gdfb['LATITUDE'])]

gdfb = GeoDataFrame(gdfb, geometry=geometry) 

## Heatmap of points (Single year)

### Emissions and Energy Variables

ENERGYSTARSCORE', 'SITEEUI_KBTU_FT', 'WEATHERNORMALZEDSITEEUI_KBTUFT',
       'SOURCEEUI_KBTU_FT', 'WEATHERNORMALZEDSOUREUI_KBTUFT',
       'TOTGHGEMISSIONS_METRICTONSCO2E', 'TOTGHGEMISSINTENSITY_KGCO2EFT',
       'WATERSCORE_MFPROPERTIES', 'WATERUSE_ALLWATERSOURCES_KGAL',
       'NATURALGASUSE_THERMS', 'FUELOILANDDIESELFUELUSEKBTU',
       'METEREDAREAS_ENERGY', 'METEREDAREAS_WATER', 'DISTRCHILLEDWATER_KBTU',
       'DISTRHOTWATER_KBTU', 'DISTRSTEAM_KBTU', 'ELECTRICITYUSE_RENEWABLE_KWH',
       'ELECTRICITYUSE_GRID_KWH',

In [ ]:

#Insert required variables, adjust opacity and blur
gdfb20=gdfb[gdfb.REPORTINGYEAR==2020]
hm20 = gdfb20[["LATITUDE", "LONGITUDE", "SITEEUI_KBTU_FT"]]
hm20=hm20.dropna()

ht_map_e = Map(location=[38.910, -77.035], zoom_start=12, tiles='openstreetmap' )

# Default: min_opacity=0.4, radius=10, blur=8
mp_e_1 = HeatMap(hm20, min_opacity=0.4, radius=10, blur=8)

ht_map_e.add_child(mp_e_1)

#insert titles and legend


## Correlation heatmaps

### How are different energy source usages correlated?

First, subset to dataframe only with only relevant energy/emissions variables.

Calculate correlation matrix. Diagonal divides it into two symmetric triangles. Keep only one.


#### Total annual energy consumption from District Hot Water 
#### highly correlated (0.65-0.75) with EUI and CO2 emissions.

In [ ]:
dfb_cor = dfb[['ENERGYSTARSCORE', 'SOURCEEUI_KBTU_FT', 'TOTGHGEMISSINTENSITY_KGCO2EFT', 
               'WATERSCORE_MFPROPERTIES', 'WATERUSE_ALLWATERSOURCES_KGAL', 'NATURALGASUSE_THERMS', 
               'FUELOILANDDIESELFUELUSEKBTU', 'METEREDAREAS_ENERGY', 'METEREDAREAS_WATER', 'DISTRCHILLEDWATER_KBTU', 

               'DISTRHOTWATER_KBTU', 'DISTRSTEAM_KBTU', 'ELECTRICITYUSE_RENEWABLE_KWH', 'ELECTRICITYUSE_GRID_KWH']]

In [ ]:
cor1= dfb_cor.corr()

# Getting the Upper Triangle of the co-relation matrix
matrix = np.triu(cor1)

# using the upper triangle matrix as mask 
ht_cor1= sns.heatmap(cor1, annot=True, mask=matrix, vmin=-1, vmax=1, cmap='Reds')
ht_cor1

## Energy/Emissions - Changes through time



In [ ]:
dfb2=dfb[dfb.REPORTINGYEAR>2012]


ENERGYSTARSCORE', 'SITEEUI_KBTU_FT', 'WEATHERNORMALZEDSITEEUI_KBTUFT', 'SOURCEEUI_KBTU_FT', 'WEATHERNORMALZEDSOUREUI_KBTUFT', 'TOTGHGEMISSIONS_METRICTONSCO2E', 'TOTGHGEMISSINTENSITY_KGCO2EFT', 'WATERSCORE_MFPROPERTIES', 'WATERUSE_ALLWATERSOURCES_KGAL', 'NATURALGASUSE_THERMS', 'FUELOILANDDIESELFUELUSEKBTU', 'METEREDAREAS_ENERGY', 'METEREDAREAS_WATER', 'DISTRCHILLEDWATER_KBTU', 'DISTRHOTWATER_KBTU', 'DISTRSTEAM_KBTU', 'ELECTRICITYUSE_RENEWABLE_KWH', 'ELECTRICITYUSE_GRID_KWH',

## Calculate changes for energy metrics for buildings from XXXX - 2021

In [ ]:
# Indicator to visualise
metric = 'TOTGHGEMISSIONS_METRICTONSCO2E'

# Comparison year (w.r.t 2021)
comp_year = 2017


In [ ]:

dfb4=dfb2.copy()


dfb4 = dfb4[dfb4.REPORTINGYEAR.isin([comp_year, 2021])]
dfb4= dfb4.dropna(subset=[metric])

#drop rows with NA of metric 
#dfb4[~(dfb4['REPORTINGYEAR']==comp_year) & df['act'].isna())]

dfb4 = dfb4[[metric, 'REPORTINGYEAR', 'PRIMARYPROPERTYTYPE_EPACALC', 'PID' ]]

dfb4=dfb4.pivot(index = ['PID', 'PRIMARYPROPERTYTYPE_EPACALC'], columns = 'REPORTINGYEAR', values = metric).reset_index()

# Calculate percent change
dfb4['del_17'] = 100*(dfb4[2021] - dfb4[comp_year])/dfb4[comp_year]
dfb4=dfb4.dropna(subset=['del_17'])

dfb4['ptype'] = dfb4['PRIMARYPROPERTYTYPE_EPACALC']
dfb4=dfb4[['PID', 'del_17', 'ptype']]

## Name columns in a loop
#for x in dfb4.columns.values:
   # dfb4["change_{0}".format(x)] = dfb4[2021] - dfb4[x]
    
    
# Merge back with main    
gdfb_21 = gdfb[gdfb.REPORTINGYEAR==2021]

gdfb_21 = pd.merge(gdfb_21, dfb4, left_on='PID', right_on = 'PID', how = 'inner')


## Map it

In [ ]:
hm_17 = gdfb_21[["LATITUDE", "LONGITUDE", "del_17"]]
#hm_17=hm_17.dropna()

ht_map_e = Map(location=[38.910, -77.035], zoom_start=12, tiles='openstreetmap' )

# Default: min_opacity=0.4, radius=10, blur=8
mp_e_1 = HeatMap(hm_17, min_opacity=0.4, radius=10, blur=8)

ht_map_e.add_child(mp_e_1)

## Descriptive statistics of changes (2017 - 2021) - PERCENT

In [ ]:

gdfb_21.loc[gdfb_21["PROPERTYNAME"] == "Georgetown University", "ptype"] = "Georgetown University"

gdfb_21=gdfb_21[gdfb_21.ptype.isin(['Multifamily Housing', 'Office', 'College/University', "Georgetown University"])]


#fig, ax = plt.subplots(figsize=(5,5))

#sns.boxplot(data=gdfb_21, x="del_17", y="ptype", hue="ptype", showfliers = False)
#plt.axvline(x = 0.0, color = 'r', linestyle = '-')
#ax.get_legend().remove()

#sns.barplot(data=gdfb_21, x="ptype", y="del_17")



## 3. Time plot - monthly (2018m1-2022m1)

### a. Make monthly dataframe

In [ ]:


dfb5 = dfb2[dfb2.REPORTINGYEAR>2017]



# Make ptype column - Georgetown as separate category
dfb5['ptype'] = dfb5['PRIMARYPROPERTYTYPE_EPACALC']
dfb5.loc[dfb5["PROPERTYNAME"] == "Georgetown University", "ptype"] = "Georgetown University"


# Make wide
dfb5= pd.wide_to_long(dfb5, stubnames=['ELECTRICITYUSE_KBTU','NATURALGAS_KBTU'] , i=['OBJECTID', 'REPORTINGYEAR', 'PID', 'SSL', 'PRIMARYPROPERTYTYPE_EPACALC'], j='month', sep='_', suffix=r'\w+').reset_index()

dfb5['REPORTINGYEAR'] = dfb5['REPORTINGYEAR'].astype(str)
dfb5['time'] = dfb5.month + "-" + dfb5.REPORTINGYEAR
dfb5['time2'] = pd.to_datetime(dfb5.time, format='%B-%Y')

#dfb5=dfb5[dfb5.ptype.isin(['Multifamily Housing', 'Office', 'College/University', "Georgetown University"])]

dfb5['REPORTINGYEAR'] = dfb5['REPORTINGYEAR'].astype(int)

dfb5[['ptype', 'month', 'REPORTINGYEAR', 'time', 'time2']].sample(5)


dfb5.head(5)

### b. Plot Electricity and Natural Gas use

In [ ]:
#dfb5=dfb5.ELECTRICITYUSE_KBTU.dropna()
#pd.set_option("display.max_rows", None)

df_gu = dfb5[(dfb5.ptype=="Georgetown University")]

dfb5=dfb5[dfb5.ptype.isin(['Multifamily Housing', 'Office', 'College/University',
                            'Mixed Use Property'])]


import seaborn as sns

sns.set_style("whitegrid")


fig, ax = plt.subplots()


#palette1 = {c:'red' if c=='Office'  else 'grey' for c in cols2.PRIMARYPROPERTYTYPE_EPACALC.unique()}


sns.lineplot(x="time2", y="ELECTRICITYUSE_KBTU",  hue="ptype", data=dfb5, legend=True, ax=ax, alpha=0.25)
sns.lineplot(x="time2", y="ELECTRICITYUSE_KBTU", color="black", data=df_gu, legend=False, ax=ax,  linestyle="dashed")
sns.set(rc={'figure.figsize':(10,10)})

fig.figure.autofmt_xdate()



## 4. Time plot - Annual

In [ ]:


dfb2['ptype']=dfb2['PRIMARYPROPERTYTYPE_EPACALC']

dfb2.loc[dfb2["PROPERTYNAME"] == "Georgetown University", "ptype"] = "Georgetown University"

dfb3 = dfb2[(dfb2.REPORTINGYEAR>2013)]


df_gu = dfb3[(dfb3.ptype=="Georgetown University")]

#dfb3=dfb3.groupby(['ptype', 'REPORTINGYEAR']).agg({"TOTGHGEMISSINTENSITY_KGCO2EFT":np.mean }).reset_index()


dfb3=dfb3[dfb3.ptype.isin(['Multifamily Housing', 'Office', 'College/University'])]

sns.set_style("whitegrid")
fig, ax = plt.subplots(figsize=(8,7))
plt.axvline(2020)

# Lineplot
sns.lineplot(x="REPORTINGYEAR", y="TOTGHGEMISSINTENSITY_KGCO2EFT",  hue="ptype", data=dfb3,
             legend=True, ax=ax, alpha=0.7, linewidth=1)

#Only Georgetown
#sns.lineplot(x="REPORTINGYEAR", y="TOTGHGEMISSINTENSITY_KGCO2EFT",  color="black", data=df_gu, 
 #            legend=True, ax=ax, alpha=0.7, linewidth=1, linestyle='dashed')

plt.title('Building Emissions by Type (2014 - 2021', fontsize=12)


## Census API data


We now pull data from the Census for Washington DC.

Specs: ACS 5 year, blockgroup level, variables - (income, racial composition)

List of available datasets : https://api.census.gov/data.html

Available APIs list: https://www.census.gov/data/developers/data-sets.html

Python Census package: https://github.com/datamade/census

PyGIS: https://pygis.io/docs/d_access_census.html

### Insert own Census API key in the cell below, assigning to variable census_api

In [ ]:
from census import Census

census_api = Census('2f1473c692f61175605ea04cbe2a9a1b41d5bf7c')

### 1. American Community Survey (.acs5)

Select variable codes from here:

 5 year ACS: https://api.census.gov/data/2020/acs/acs5/variables.html
 
 https://api.census.gov/data/2019/acs/acs5/variables.html
 
 Example (non-code): https://www.census.gov/content/dam/Census/library/publications/2020/acs/acs_general_handbook_2020.pdf
 
 
 ### Decennial Census - 2010 (.sf1)

The ACS 2020 has many key variables missing at the block group level. Next try the Decennial Census 2020.

The base way to call the Census API is given here.
https://api.census.gov/data/2020/dec/pl/examples.html

The Python Census package can only call data for SF1 (summary file, 2011 census).


#### Geo: block-group
    *Var*: HOUSEHOLD INCOME IN THE LAST 12 MONTHS (IN 2019 INFLATION-ADJUSTED DOLLARS), race

In [ ]:

# income, race
acs5_19_bg = census_api.acs5.state_county_blockgroup(fields = ('NAME', 'B19013_001E',  'B02001_001E', 'B02001_002E', 'B02001_003E'),
                                      state_fips = 11,
                                      county_fips = "*",
                                      blockgroup = "*",
                                      year = 2019)

#convert to pandas df
acs5_19_bg = pd.DataFrame(acs5_19_bg)

# Create new variables with better names
acs5_19_bg['white_pop'] = acs5_19_bg.B02001_002E
acs5_19_bg['black_pop'] = acs5_19_bg.B02001_003E
acs5_19_bg['tot_pop'] = acs5_19_bg.B02001_001E
acs5_19_bg['median_inc'] = acs5_19_bg.B19013_001E

acs5_19_bg = acs5_19_bg[['NAME', 'white_pop', 'black_pop', 'tot_pop', 'median_inc', 'state', 'county', 'tract', 'block group']]


acs5_19_bg.head()

## Call [block group] Shapefile, merge with data 


All US geographies and shapefiles: https://www.census.gov/geographies/mapping-files/time-series/geo/cartographic-boundary.2021.html#list-tab-WOE0F8BPHOHJO0GYNN



In [ ]:
#host = 'https:/www2.census.gov'

dc_bg_19 = gpd.read_file('https://www2.census.gov/geo/tiger/TIGER2019/BG/tl_2019_11_bg.zip')


dc_bg_19['id']=dc_bg_19['GEOID']


acs5_19_bg['id'] = acs5_19_bg.state.astype(str) + acs5_19_bg.county.astype(str) + acs5_19_bg.tract.astype(str) + acs5_19_bg['block group'].astype(str) 

dc_bg_19 = pd.merge(dc_bg_19, acs5_19_bg, left_on='id', right_on='id', how='left')

In [ ]:
dc_bg_19.columns

### Create variables and map





In [ ]:
dc_bg_19=dc_bg_19[dc_bg_19.median_inc>0]

dc_bg_19['bl_perc'] = dc_bg_19.black_pop/dc_bg_19.tot_pop

dc_bg_19['w_perc'] = dc_bg_19.white_pop/dc_bg_19.tot_pop


In [ ]:
fig, ax = plt.subplots(figsize=(8,9))

plt.title('Median Income', fontsize=20)

dc_bg_19.plot(ax=ax, column='median_inc', legend=True)

### Blockgroups with higher share of black population have lower median income

In [ ]:
import plotly.express as px

fig1 = px.scatter(dc_bg_19, x='bl_perc', y='median_inc', color='w_perc', opacity = .4, 
                  trendline="lowess", trendline_options=dict(frac=0.4))

#fig1 = px.scatter(dc_bg_19, x='bl_perc', y='median_inc', color='w_perc', opacity = .4, 
 #    trendline="ols")
fig1

## Merge: performance + benchmark, 2019

Performance is reported only for 2019. 

To merge, subset benchmarking data to only 2019.

Use SSL as key for merging. Gives highest number of matches (1922)


In [ ]:
dfbp19 = dfb[dfb.REPORTINGYEAR==2019]
dfbp19.shape

dfbp19 = pd.merge(dfbp19, dfp, left_on='SSL', right_on='SSL',
                  how='inner')

dfbp19.shape

## Overlay energy data - building points (benchmark + BEPS), 2019

In [ ]:
geometry = [Point(xy) for xy in zip(dfbp19['LONGITUDE_y'], dfbp19['LATITUDE_y'])]

dfbp19 = GeoDataFrame(dfbp19, geometry=geometry) 

dfbp19=pd.get_dummies(dfbp19, columns=['MEETS_BEPS'])



In [ ]:
dfbp19.crs='epsg:4326'
dc_bg_19.crs='epsg:4326'

In [ ]:

dfbp19['meets_beps'] = dfbp19['MEETS_BEPS_Meets BEPS']
dfbp19['n_build'] =  1             

dfbp19['meets_beps'].value_counts()

In [ ]:

fig, ax = plt.subplots(figsize=(10,8))

dc_bg_19.plot(ax=ax, facecolor='Grey', edgecolor='k',alpha=1,linewidth=0.3, cmap='binary', column='median_inc', legend=True)

dfbp19_2 = dfbp19[dfbp19.meets_beps==0]
dfbp19_2.plot(color='red', ax=ax, markersize=2, marker='o', alpha=0.8)


#You can use different 'cmaps' such as jet, plasm,magma, infereno,cividis, binary...(I simply chose cividis)


#plt.title('Buildings that Failed BEPS by Block Group Median Income', fontsize=12)
ax.set_xlabel('Longitude', fontsize=10)
ax.set_ylabel('Latitude', fontsize=10)


## Spatial Join

In [ ]:
# Assign each building point in Energy data to a blockgroup

map_pt = gpd.sjoin(dfbp19, dc_bg_19, predicate='intersects') 

#print(list(map_pt.columns))

map_pt.bl_perc = 100*map_pt.bl_perc

## Logistic regression - Meets Beps on area income, demogr

Run a logistic regression on building level data. 

Dependent variable = Building meets BEPS (1 if true, 0 if false)

Regressors = [Bulding site EUI, BlkGrp % black, BlkGrp Median income]

Standard errors clustered by block group (grouping variable 'id')

Maximum Likelihood optimisation algorithm: Newton-Raphson

###


In [ ]:
import statsmodels.formula.api as smf

reg_df1 = map_pt[['TAXRECORDFLOORAREA_x', 'SITEEUI_KBTU_FT', 'bl_perc', 'median_inc', 'meets_beps', 'id']]

reg_df1=reg_df1.dropna()

#define response variable
y = reg_df1['meets_beps']

#define predictor variables
x = reg_df1[['SITEEUI_KBTU_FT', 'bl_perc', 'median_inc']]

log_reg = smf.logit("meets_beps ~ SITEEUI_KBTU_FT + bl_perc + median_inc",
                    data=reg_df1).fit(cov_type='cluster', method='newton', cov_kwds={'groups': reg_df1['id']})

print(log_reg.summary())

print(log_reg.get_margeff(at ='overall').summary()) # get marginal effects


## Results (in Odd ratios)

### A building in a block group with 1% more minority population was 0.6% less likely to meet performance standards in 2019.
### A 1 kBTU/Ft rise in Site EUI of building associated with a 2.3% lower chance of passing.

In [ ]:
model_odds = pd.DataFrame(np.exp(log_reg.params), columns= ['OR'])
model_odds['z-value']= log_reg.pvalues
model_odds[['2.5%', '97.5%']] = np.exp(log_reg.conf_int())

model_odds

## Group to block level

In [ ]:

map_pt = map_pt.groupby(['id']).agg(bl_perc=('bl_perc', np.mean), 
                                            med_inc=('median_inc', np.mean),
                                            wh_perc = ('w_perc', np.mean),
                                            av_st_eui = ('SITEEUI_KBTU_FT', np.mean),
                                            meets_bep = ('meets_beps', np.sum),
                                            tot_bldn = ('n_build', np.sum)).reset_index()

map_pt['prop_meet_beps'] = map_pt.meets_bep/map_pt.tot_bldn

In [ ]:

import statsmodels.api as sm

#define response variable
y = df2['av_st_eui']

#define predictor variables
x = df2[['bl_perc', 'med_inc']]

#add constant to predictor variables
x = sm.add_constant(x)

#fit linear regression model
model = sm.OLS(y, x).fit()

#view model summary
print(model.summary())


## Just a plot
fig2 = px.scatter(map_pt, x='bl_perc', y='prop_meet_beps', color='tot_bldn', opacity = .4, 
                  trendline="ols")


### Geo: Tract
   *Var*: Median monthly housing costs, race

In [ ]:

# income, race
acs5_19_ct = census_api.acs5.state_county_tract(fields = ('NAME', 'B25105_001E', 'B19013_001E', 'B19013A_001E', 'B19013B_001E', 'B02001_001E', 'B02001_002E', 'B02001_003E', 'B19049_001E'),
                                      state_fips = 11,
                                      county_fips = "*",
                                      tract = "*",
                                      year = 2019)

#convert to pandas df
acs5_19_ct = pd.DataFrame(acs5_19_ct)


# Create new variables with better names
acs5_19_ct['white_pop'] = acs5_19_ct.B02001_002E
acs5_19_ct['black_pop'] = acs5_19_ct.B02001_003E
acs5_19_ct['tot_pop'] = acs5_19_ct.B02001_001E
acs5_19_ct['median_inc'] = acs5_19_ct.B19013_001E
acs5_19_ct['median_inc_wh'] = acs5_19_ct.B19013A_001E
acs5_19_ct['median_inc_bl'] = acs5_19_ct.B19013B_001E
acs5_19_ct['median_hous_cost'] = acs5_19_ct.B02001_003E


acs5_19_ct = acs5_19_ct[['NAME', 'white_pop', 'black_pop', 'tot_pop', 'median_inc', 'median_inc_wh', 'median_inc_bl', 'median_hous_cost', 'state', 'county', 'tract']]


acs5_19_ct.head()

# Next steps:
- 
